In [ ]:
!pip install pyrankvote

In [63]:
import pyrankvote
import ast
import pandas as pd
from pyrankvote import Candidate, Ballot


def rename_index_col_to_ballot_id(df):
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
    return df

def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x != '0', list_of_unique_vals_in_dataframe))
    return cand_list

def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]

# def initialize_cand_objs_in_df_cols(df):
#     cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
#     cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
#     cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
#     return cands_df

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:, 1:].agg(", ".join, axis=1)
    # cands_to_put_in_single_cell = df.iloc[:, 1:].columns.to_list()
    # df['candidate_list'] = str(cands_to_put_in_single_cell)
    # df['candidate_list'] = df['candidate_list'].apply(lambda x: ast.literal_eval(x))
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects

def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']

def run_all_steps(df):
    election_dict = {}
    #df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename'], axis=1)
    print('df.shape:',df.shape)
    df = get_cands_into_single_cell(df)

    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
    df = rm_invalid_rows(df)
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots
    
    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
    print("cand_list: ",cand_list)
#     print(df.head())
    election = run_election(cand_list, df)
    winner = election.get_winners()
    print("get_winners",winner)
    election_dict['winner'] = winner
    #df["irv_winner"] = winner
    print("rounds",len(election.rounds))
    election_dict['rounds'] = len(election.rounds)

#     open file and write results. read winners votes from file and write to df
    #print("register_round_results",election.register_round_results(election.rounds[-1]))

    print("election:", election)
    #election_dict['election'] = election

#     print("election type:", type(election))
    #election_dict['register']=election.register_round_results(election.rounds[-1])

    #print('hi')
    return df, election_dict

def read_votes(file, election_dict):
    with open(file, 'r') as file1:
        count = 0
        while True: 
            count += 1

            # Get next line from file 
            line = file1.readline()
            if not line: 
                break
            strip_line = line.strip()
    #         print('*',strip_line,'*')
            if 'ROUND' in strip_line or 'FINAL' in strip_line:
                start_round = True
                if 'ROUND 1' in strip_line or 'FINAL' in strip_line:
                    monitor_round=True
                    print('*',strip_line,'*')
                else:
                    monitor_round=False
            if 'ROUND' not in strip_line and  'FINAL' not in strip_line:
                start_round = False
                if winner in line and monitor_round:
                    print('*',strip_line,'*')
                    if 'firstRoundVotes' not in election_dict:
                        election_dict['firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    else:
                        election_dict['finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    
    file1.close() 
    






In [51]:
df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
df = rename_index_col_to_ballot_id(df)
# df['irv_winner']=""
df = df[(df['filename']!="election_05-04-2020_08-39-40_5cands_12noise.csv") | (df['ballot_id']!=2767)]
df = df[(df['filename']!="election_04-30-2020_21-07-57_4_0.01.csv") | (df['ballot_id']!=20726)]
master_df = df.copy(deep=True)

/Users/shvarugh/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('max_colwidth', 8000)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_rows = 10000

In [15]:
election_ids= df['filename'].unique()
#election_ids

In [23]:
all_elections_dict = {}

In [17]:
df.head()

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,candidate_6,filename
0,0,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
1,1,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
2,2,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
3,3,candidate_1,candidate_2,candidate_3,0,0,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
4,4,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv


In [64]:
election_num_dict = {}
for election_num, election in enumerate(election_ids):
    election_num_dict[election_num] = election
    print(election_num, election)

0 election_05-04-2020_08-40-23_5cands_16noise.csv
1 election_04-30-2020_21-00-03_4_0.0225.csv
2 election_05-02-2020_08-18-17_5cands_3noise.csv
3 election_05-04-2020_08-38-34_4cands_12noise.csv
4 election_04-30-2020_20-59-34_4_0.0375.csv
5 election_04-30-2020_21-06-11_4_0.0125.csv
6 election_04-30-2020_21-08-19_3_0.013333333333333334.csv
7 election_05-04-2020_08-40-33_4cands_10noise.csv
8 election_05-02-2020_08-18-31_3cands_18noise.csv
9 election_04-30-2020_21-04-28_6_0.006666666666666667.csv
10 election_04-30-2020_21-08-00_4_0.0275.csv
11 election_05-04-2020_08-39-40_5cands_12noise.csv
12 election_04-30-2020_21-07-57_4_0.01.csv
13 election_04-30-2020_21-08-54_3_0.02.csv
14 election_05-02-2020_08-19-42_3cands_2noise.csv
15 election_05-04-2020_08-40-11_5cands_8noise.csv
16 election_05-04-2020_08-40-56_5cands_17noise.csv
17 election_05-02-2020_08-17-43_3cands_11noise.csv
18 election_05-02-2020_08-19-53_3cands_16noise.csv
19 election_05-04-2020_08-41-05_4cands_3noise.csv
20 election_05-02-

Check:
Election 11,12 failed

In [50]:
df[df['filename']=="election_04-30-2020_21-07-57_4_0.01.csv"]

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,candidate_6,filename
296340,0,candidate_1,candidate_2,candidate_3,candidate_4,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
296341,1,candidate_1,candidate_2,candidate_3,candidate_4,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
296342,2,candidate_1,candidate_2,candidate_3,candidate_4,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
296343,3,candidate_1,candidate_2,candidate_3,candidate_4,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
296344,4,candidate_1,candidate_2,candidate_3,candidate_4,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
...,...,...,...,...,...,...,...,...
317062,20722,candidate_4,candidate_3,candidate_2,candidate_1,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
317063,20723,candidate_4,candidate_3,candidate_2,candidate_1,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
317064,20724,candidate_4,candidate_3,candidate_2,candidate_1,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv
317065,20725,candidate_4,candidate_3,candidate_2,candidate_1,NaN,NaN,election_04-30-2020_21-07-57_4_0.01.csv


In [58]:
election_dict = {}
#df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
df = df[df['filename']=="election_05-04-2020_08-39-40_5cands_12noise.csv"]
df.to_csv('election11.csv', index = False, header=True)
df = df.dropna(axis=1, how='all')
df = df.drop(['ballot_id','filename'], axis=1)
print('df.shape:',df.shape)
df = get_cands_into_single_cell(df)

df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
df = rm_invalid_rows(df)
df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

ballots = initialize_ballot_objs(df)
df['ballots'] = ballots

cand_list = get_cand_list(df)
cand_list = initialize_cand_objs(cand_list)
print("cand_list: ",cand_list)
#     print(df.head())
election = run_election(cand_list, df)
winner = election.get_winners()
print("get_winners",winner)
election_dict['winner'] = winner
#df["irv_winner"] = winner
print("rounds",len(election.rounds))
election_dict['rounds'] = len(election.rounds)

#     open file and write results. read winners votes from file and write to df
#print("register_round_results",election.register_round_results(election.rounds[-1]))

print("election:", election)
election_dict['election'] = election

#     print("election type:", type(election))
#election_dict['register']=election.register_round_results(election.rounds[-1])

#print('hi')
# return df, election_dict

df.shape: (2767, 5)
cand_list:  [<Candidate('candidate_1')>, <Candidate('candidate_4')>, <Candidate('candidate_3')>, <Candidate('candidate_2')>, <Candidate('candidate_5')>]
get_winners [<Candidate('candidate_3')>]
rounds 4
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1      593  Hopeful
candidate_4      583  Hopeful
candidate_3      565  Hopeful
candidate_2      539  Hopeful
candidate_5      487  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_3      737  Hopeful
candidate_4      722  Hopeful
candidate_1      692  Hopeful
candidate_2      616  Rejected
candidate_5        0  Rejected

ROUND 3
Candidate      Votes  Status
-----------  -------  --------
candidate_3      988  Hopeful
candidate_4      921  Hopeful
candidate_1      858  Rejected
candidate_2        0  Rejected
candidate_5        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_3     1453  Elected
c

In [52]:
for election_num, election in enumerate(election_ids):
#     election_dict[election]=
    #print(election_num, election)
#     if election_num ==11:
    print("***********Election: ", election_num)
    df_election = df.loc[df['filename']==election]
    #print(df_election.shape)

    temp_df,election_dict = run_all_steps(df_election)
    #print(election_dict)
    if len(election_dict['winner'])==1:
        winner = election_dict['winner'][0].name
        election_dict['winner'] = winner
    else:
        print(election, "has more than one winner")
    result_file = 'out.txt'
    with open(result_file, 'w') as file1:
        print(election_dict['election'], file=file1)
    read_votes(result_file, election_dict)
    all_elections_dict[election_num] = election_dict


***********Election:  0
get_winners [<Candidate('candidate_1')>]
rounds 4
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1     9129  Hopeful
candidate_5     9010  Hopeful
candidate_2     8455  Hopeful
candidate_3     7911  Hopeful
candidate_4     6858  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_1    10903  Hopeful
candidate_5    10640  Hopeful
candidate_3    10078  Hopeful
candidate_2     9742  Rejected
candidate_4        0  Rejected

ROUND 3
Candidate      Votes  Status
-----------  -------  --------
candidate_1    14071  Hopeful
candidate_3    13673  Hopeful
candidate_5    13619  Rejected
candidate_2        0  Rejected
candidate_4        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_1    20711  Elected
candidate_3    20652  Rejected
candidate_5        0  Rejected
candidate_2        0  Rejected
candidate_4        0  Rejected

* ROUND 1 *
* candidate_1

get_winners [<Candidate('candidate_4')>]
rounds 3
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_4    10258  Hopeful
candidate_3     9745  Hopeful
candidate_2     8003  Hopeful
candidate_1     5309  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_3    12505  Hopeful
candidate_4    11395  Hopeful
candidate_2     9415  Rejected
candidate_1        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_4    17129  Elected
candidate_3    16186  Rejected
candidate_2        0  Rejected
candidate_1        0  Rejected

* ROUND 1 *
* candidate_4    10258  Hopeful *
* FINAL RESULT *
* candidate_4    17129  Elected *
***********Election:  11
get_winners [<Candidate('candidate_3')>]
rounds 4
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_1      593  Hopeful
candidate_4      583  Hopeful
candidate_3      565  Hopeful
candidate_2      539  

get_winners [<Candidate('candidate_3')>]
rounds 3
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_3     5418  Hopeful
candidate_2     4518  Hopeful
candidate_1     3443  Hopeful
candidate_4     3184  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
candidate_3     6730  Hopeful
candidate_2     4996  Hopeful
candidate_1     4837  Rejected
candidate_4        0  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_3     9074  Elected
candidate_2     7489  Rejected
candidate_1        0  Rejected
candidate_4        0  Rejected

* ROUND 1 *
* candidate_3     5418  Hopeful *
* FINAL RESULT *
* candidate_3     9074  Elected *
***********Election:  22
get_winners [<Candidate('candidate_4')>]
rounds 3
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_4     3381  Hopeful
candidate_1     2748  Hopeful
candidate_3     2411  Hopeful
candidate_2     2008  

In [59]:
all_elections_dict

{0: {'winner': 'candidate_1',
  'rounds': 4,
  'election': <ElectionResults(5 rounds)>,
  'register': None,
  'firstRoundVotes': 9129,
  'finalRoundVotes': 20711},
 1: {'winner': 'candidate_1',
  'rounds': 3,
  'election': <ElectionResults(4 rounds)>,
  'register': None,
  'firstRoundVotes': 13918,
  'finalRoundVotes': 22940},
 2: {'winner': 'candidate_4',
  'rounds': 4,
  'election': <ElectionResults(5 rounds)>,
  'register': None,
  'firstRoundVotes': 7502,
  'finalRoundVotes': 17361},
 3: {'winner': 'candidate_1',
  'rounds': 3,
  'election': <ElectionResults(4 rounds)>,
  'register': None,
  'firstRoundVotes': 3400,
  'finalRoundVotes': 5758},
 4: {'winner': 'candidate_2',
  'rounds': 3,
  'election': <ElectionResults(4 rounds)>,
  'register': None,
  'firstRoundVotes': 11236,
  'finalRoundVotes': 24975},
 5: {'winner': 'candidate_4',
  'rounds': 3,
  'election': <ElectionResults(4 rounds)>,
  'register': None,
  'firstRoundVotes': 2637,
  'finalRoundVotes': 4767},
 6: {'winner': '

In [67]:
election_results_df=pd.DataFrame()
election_results_list=[]

In [65]:
election_num_dict[1]

'election_04-30-2020_21-00-03_4_0.0225.csv'

In [68]:
for key,val in all_elections_dict.items():
    print(val)
    val['election_id']=election_num_dict[key]
    #del val['register']
    #del val['election']
    
    election_results_list.append(val)
    

{'winner': 'candidate_1', 'rounds': 4, 'firstRoundVotes': 9129, 'finalRoundVotes': 20711, 'election_id': 'election_05-04-2020_08-40-23_5cands_16noise.csv'}
{'winner': 'candidate_1', 'rounds': 3, 'firstRoundVotes': 13918, 'finalRoundVotes': 22940}
{'winner': 'candidate_4', 'rounds': 4, 'firstRoundVotes': 7502, 'finalRoundVotes': 17361}
{'winner': 'candidate_1', 'rounds': 3, 'firstRoundVotes': 3400, 'finalRoundVotes': 5758}
{'winner': 'candidate_2', 'rounds': 3, 'firstRoundVotes': 11236, 'finalRoundVotes': 24975}
{'winner': 'candidate_4', 'rounds': 3, 'firstRoundVotes': 2637, 'finalRoundVotes': 4767}
{'winner': 'candidate_3', 'rounds': 2, 'firstRoundVotes': 8199, 'finalRoundVotes': 9844}
{'winner': 'candidate_3', 'rounds': 3, 'firstRoundVotes': 8907, 'finalRoundVotes': 17218}
{'winner': 'candidate_2', 'rounds': 2, 'firstRoundVotes': 2034, 'finalRoundVotes': 3069}
{'winner': 'candidate_3', 'rounds': 5, 'firstRoundVotes': 4260, 'finalRoundVotes': 12655}
{'winner': 'candidate_4', 'rounds': 

In [69]:
election_results_list

[{'winner': 'candidate_1',
  'rounds': 4,
  'firstRoundVotes': 9129,
  'finalRoundVotes': 20711,
  'election_id': 'election_05-04-2020_08-40-23_5cands_16noise.csv'},
 {'winner': 'candidate_1',
  'rounds': 3,
  'firstRoundVotes': 13918,
  'finalRoundVotes': 22940,
  'election_id': 'election_04-30-2020_21-00-03_4_0.0225.csv'},
 {'winner': 'candidate_4',
  'rounds': 4,
  'firstRoundVotes': 7502,
  'finalRoundVotes': 17361,
  'election_id': 'election_05-02-2020_08-18-17_5cands_3noise.csv'},
 {'winner': 'candidate_1',
  'rounds': 3,
  'firstRoundVotes': 3400,
  'finalRoundVotes': 5758,
  'election_id': 'election_05-04-2020_08-38-34_4cands_12noise.csv'},
 {'winner': 'candidate_2',
  'rounds': 3,
  'firstRoundVotes': 11236,
  'finalRoundVotes': 24975,
  'election_id': 'election_04-30-2020_20-59-34_4_0.0375.csv'},
 {'winner': 'candidate_4',
  'rounds': 3,
  'firstRoundVotes': 2637,
  'finalRoundVotes': 4767,
  'election_id': 'election_04-30-2020_21-06-11_4_0.0125.csv'},
 {'winner': 'candidate_

In [71]:
election_results_df = pd.DataFrame(election_results_list)
election_results_df.to_csv('election_results.csv', index=False)